# Set Up Databases and Populating Tickers and Historical Stock Data

# 1. Setup & Imports

In [1]:
import sys
import os
import sqlite3
from pathlib import Path
from datetime import datetime, timedelta

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

# Import paths from the config module
from src.config import BASE_DIR, DB_DIR

from src.db_schema import DATABASES
from src.etl.populate_tickers import recreate_database, populate_tickers
from src.etl.populate_prices import populate_prices
from src.utils.db_utils import fetch_all_asset_metadata, fetch_all_asset_prices

Project root added to sys.path: /home/aaronnhorvitz/dev/school/stat_656_autotrader


# 2. Initialize SQLite Databases

In [2]:
DB_DIR.mkdir(exist_ok=True)

def setup_databases():
    """Initialize databases and tables based on defined schemas."""
    for db_name, schemas in DATABASES.items():
        db_path = DB_DIR / db_name

        try:
            with sqlite3.connect(db_path) as conn:
                conn.execute("PRAGMA foreign_keys = ON;")
                cursor = conn.cursor()
                for schema in schemas:
                    cursor.execute(schema)
                conn.commit()
                print(f"✅ Tables created/verified in '{db_name}'")
        except sqlite3.Error as e:
            print(f"❌ Error setting up '{db_name}': {e}")

# Run database setup
setup_databases()

✅ Tables created/verified in 'assets.db'
✅ Tables created/verified in 'portfolio_management.db'
✅ Tables created/verified in 'accounting.db'
✅ Tables created/verified in 'modeling.db'
✅ Tables created/verified in 'exogenous.db'
✅ Tables created/verified in 'db_change_log.db'


# 3. Populate Asset Metadata (Tickers)

In [3]:
print("\n🔄 Recreating and populating tickers database...")
recreate_database()
populate_tickers()
print("✅ Tickers populated successfully.")


🔄 Recreating and populating tickers database...
✅ Tickers populated successfully.


In [4]:
# View the asset metadata
asset_metadata_df = fetch_all_asset_metadata()

print("DataFrame shape:", asset_metadata_df.shape)
print("DataFrame columns:", asset_metadata_df.columns.tolist())
display(asset_metadata_df.head(20))

Connecting to database: /home/aaronnhorvitz/dev/school/stat_656_autotrader/databases/assets.db
Raw rows fetched: [(1, 'MTEN', 'Mingteng International Corporation Inc. Ordinary Shares', 'NASDAQ', 'us_equity', 1, '2025-04-28', None, '2025-04-28 15:05:38'), (2, 'THIR', 'THOR Index Rotation ETF', 'NYSE', 'us_equity', 1, '2025-04-28', None, '2025-04-28 15:05:38'), (3, 'NFBK', 'Northfield Bancorp, Inc. Common Stock (Delaware)', 'NASDAQ', 'us_equity', 1, '2025-04-28', None, '2025-04-28 15:05:38'), (4, 'NEXN', 'Nexxen International Ltd. Ordinary Shares', 'NASDAQ', 'us_equity', 1, '2025-04-28', None, '2025-04-28 15:05:38'), (5, 'NEXA', 'Nexa Resources S.A. Common Shares', 'NYSE', 'us_equity', 1, '2025-04-28', None, '2025-04-28 15:05:38')]
Column names from DB: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_added', 'date_removed', 'fetched_at']
DataFrame shape: (7465, 9)
DataFrame columns: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_ad

,asset_id,symbol,name,exchange,asset_type,is_active,date_added,date_removed,fetched_at
0,1,MTEN,Mingteng International Corporation Inc. Ordina...,NASDAQ,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
1,2,THIR,THOR Index Rotation ETF,NYSE,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
2,3,NFBK,"Northfield Bancorp, Inc. Common Stock (Delaware)",NASDAQ,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
3,4,NEXN,Nexxen International Ltd. Ordinary Shares,NASDAQ,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
4,5,NEXA,Nexa Resources S.A. Common Shares,NYSE,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
5,6,AAM,AA Mission Acquisition Corp.,NYSE,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
6,7,NEWZ,EA Series Trust StockSnips AI-Powered Sentimen...,NASDAQ,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
7,8,MTEKW,Maris-Tech Ltd. Warrants,NASDAQ,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
8,9,HLVX,"HilleVax, Inc. Common Stock",NASDAQ,us_equity,1,2025-04-28,None,2025-04-28 15:05:38
9,10,NEWTZ,"NewtekOne, Inc. 5.50% Notes Due 2026",NASDAQ,us_equity,1,2025-04-28,None,2025-04-28 15:05:38


In [5]:
asset_metadata_df['exchange'].unique()

array(['NASDAQ', 'NYSE', 'AMEX'], dtype=object)

# 4. Populate Historical Stock Prices

In [6]:
# Ensure the historical prices are populated. 
populate_prices()
print("✅ Historical stock prices populated successfully.")

Connecting to database: /home/aaronnhorvitz/dev/school/stat_656_autotrader/databases/assets.db
Connecting to database: /home/aaronnhorvitz/dev/school/stat_656_autotrader/databases/assets.db
Connecting to database: /home/aaronnhorvitz/dev/school/stat_656_autotrader/databases/assets.db
Connecting to database: /home/aaronnhorvitz/dev/school/stat_656_autotrader/databases/assets.db


Alpaca Download Progress: 100%|██████████| 7465/7465 [2:28:53<00:00,  1.20s/it]  


Processed 7465 tickers, missing data for 27 tickers.
Total time: 8933.26 seconds. Avg time per ticker: 1.20 seconds.
✅ Historical stock prices populated successfully.


In [7]:
# Fetch and display the historical prices
print("\n📊 Fetching and displaying historical stock prices...")
prices_df = fetch_all_asset_prices()

# Display basic info and the first 20 rows
print(f"DataFrame shape: {prices_df.shape}")
print(f"Columns: {prices_df.columns.tolist()}")
display(prices_df.head(20))


📊 Fetching and displaying historical stock prices...
Connecting to database: /home/aaronnhorvitz/dev/school/stat_656_autotrader/databases/assets.db
DataFrame shape: (6823590, 11)
Columns: ['price_id', 'asset_id', 'symbol', 'date', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'fetched_at']


,price_id,asset_id,symbol,date,open,high,low,close,adjusted_close,volume,fetched_at
0,265439,301,VALE,2017-08-29,10.67,10.67,10.67,10.67,None,None,2025-04-28 15:11:36
1,4650670,5444,CASH,2017-10-18,81.20,81.20,81.20,81.20,None,None,2025-04-28 16:53:13
2,5740297,6423,MAT,2017-10-23,15.40,15.40,15.40,15.40,None,None,2025-04-28 17:13:13
3,6098584,6745,PAC,2017-11-03,94.65,94.65,94.65,94.65,None,None,2025-04-28 17:19:54
4,5339641,6059,HHH,2017-11-14,120.61,120.61,120.61,120.61,None,None,2025-04-28 17:05:49
5,5129923,5871,FFIV,2017-11-15,120.74,120.74,120.74,120.74,None,None,2025-04-28 17:01:58
6,3880782,4749,CAR,2017-12-26,44.43,44.43,44.43,44.43,None,None,2025-04-28 16:39:01
7,6382617,6998,SABR,2017-12-26,20.54,20.54,20.54,20.54,None,None,2025-04-28 17:25:09
8,3980572,4840,PSX,2018-01-02,101.66,101.66,101.66,101.66,None,None,2025-04-28 16:40:52
9,6347817,6966,RPM,2018-01-18,52.46,52.46,52.46,52.46,None,None,2025-04-28 17:24:30
